<a href="https://colab.research.google.com/github/albeeertomoreno/03-MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos_TrabajoPr%C3%A1ctico_Alberto_Moreno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Alberto Moreno Torreira  <br>
Url: https://github.com/albeeertomoreno/03-MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos_TrabajoPr%C3%A1ctico_Alberto_Moreno.ipynb<br>
Google Colab:https://colab.research.google.com/drive/1-mjY_xoUJUvHpRdeKIQNfQygKn92lckN?usp=sharing <br>
Problema:
>1. Sesiones de doblaje

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Antes de poder desarrollar el código es necesario instalar la librería que vamos a utilizar.

In [9]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

Antes de buscar el código que minimice los costes de los actores por desplazarse al rodaje, cargamos los datos desde el Drive.

In [10]:
# Permitir acceso al drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


# Especifica la ruta en tu unidad de Google Drive al archivo Excel
ruta = '/content/drive/My Drive/Colab_Notebooks/AG_OPT/res/datos_practica1.xlsx'

nombres_columnas = ['Toma', 'actor1', 'actor2', 'actor3', 'actor4', 'actor5',
                    'actor6', 'actor7', 'actor8', 'actor9', 'actor10', 'NaN',
                    'Total']


# Lee el archivo Excel en un DataFrame de Pandas
datos_lista = pd.read_excel(ruta,names=nombres_columnas )
# Elimina la primera fila del DataFrame
datos_lista.drop([0,31,32], axis=0,inplace=True)
del datos_lista['NaN']

display(datos_lista)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Toma,actor1,actor2,actor3,actor4,actor5,actor6,actor7,actor8,actor9,actor10,Total
1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5
2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3
3,3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3
4,4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4
5,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3
6,6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4
7,7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4
8,8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
9,9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
10,10,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4


In [11]:
##Respuesta
from ortools.linear_solver import pywraplp

# Datos de entrada
n_tomas = 30  # Número total de tomas
n_actores = 10  # Número total de actores
max_tomas_por_dia = 6  # Máximo número de tomas que se pueden grabar por día
n_dias = n_tomas  # Máximo teórico de días, podría ser menor

# Convertir el DataFrame datos a una lista de listas para las restricciones de actores
datos = datos_lista.iloc[:, 1:-1].values.tolist()

# Crear el modelo de optimización
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
x = {}  # x[toma][dia] = 1 si la toma se realiza en ese día
y = {}  # y[actor][dia] = 1 si el actor trabaja ese día
for toma in range(n_tomas):
    for dia in range(n_dias):
        x[toma, dia] = solver.IntVar(0, 1, f'x[{toma},{dia}]')
for actor in range(n_actores):
    for dia in range(n_dias):
        y[actor, dia] = solver.IntVar(0, 1, f'y[{actor},{dia}]')

# Restricciones
# No más de max_tomas_por_dia tomas por día
for dia in range(n_dias):
    solver.Add(solver.Sum(x[toma, dia] for toma in range(n_tomas)) <= max_tomas_por_dia)

# Cada toma se asigna al menos una vez
for toma in range(n_tomas):
    solver.Add(solver.Sum(x[toma, dia] for dia in range(n_dias)) == 1)

# Si una toma se realiza en un día, todos los actores necesarios deben estar presentes
for toma in range(n_tomas):
    for dia in range(n_dias):
        actores_necesarios = [actor for actor in range(n_actores) if datos[toma][actor] == 1]
        for actor in actores_necesarios:
            solver.Add(x[toma, dia] <= y[actor, dia])
# Función objetivo: minimizar el número total de días de trabajo de todos los actores
solver.Minimize(solver.Sum(y[actor, dia] for actor in range(n_actores) for dia in range(n_dias)))

# Resolver el modelo
status = solver.Solve()

# Verificar si se encontró una solución óptima
if status == pywraplp.Solver.OPTIMAL:
    print('Solución:')
    for dia in range(n_dias):
        print(f'Día {dia + 1}:', end=' ')
        for toma in range(n_tomas):
            if x[toma, dia].solution_value() > 0.5:
                print(f'Toma {toma + 1}', end=' ')
        print()
    print('Número total de días de trabajo de los actores:', solver.Objective().Value())
else:
    print('No se encontró una solución óptima.')

No se encontró una solución óptima.


El método planteado de programación lineal entera resulta muy costoso por lo que buscaremos un método heurístico que resuelva el problema. Aunque la solución no sea la óptima podemos acercarnos a ella mediante la búsqueda tabú, planteando la primera solución mediante la técnica voraz.

In [12]:
from copy import deepcopy

# Renombrar el dataframe
df = datos_lista

class PlanificacionTomas:
    def __init__(self, df, max_tomas_por_dia, max_iteraciones, tamaño_tabu):
        # Inicialización de la clase con el DataFrame, el máximo de tomas por día,
        # el número máximo de iteraciones y el tamaño de la lista tabú.
        self.df = df
        self.max_tomas_por_dia = max_tomas_por_dia
        self.max_iteraciones = max_iteraciones
        self.tamaño_tabu = tamaño_tabu
        self.tabu_list = []

    def formato_solucion(self, solucion):
        # Formatea la solución para imprimir el número total de días y las tomas por día.
        resultado = f"Número total de días de grabación: {len(solucion)}\n"
        for i, dia in enumerate(solucion, 1):
            tomas_dia = ", ".join([str(toma) for toma in dia])
            resultado += f"Día {i}: Tomas {tomas_dia}\n"
        return resultado

    def enfoque_voraz(self):
        # Genera una solución inicial utilizando un enfoque voraz, ordenando las tomas por el
        # total de actores y asignándolas a los días hasta alcanzar el máximo permitido.
        dias = []
        tomas_ordenadas = self.df.sort_values(by='Total', ascending=False)['Toma'].tolist()
        for toma in tomas_ordenadas:
            agregado = False
            for dia in dias:
                if len(dia) < self.max_tomas_por_dia:
                    dia.append(toma)
                    agregado = True
                    break
            if not agregado:
                dias.append([toma])
        return dias

    def función_evaluación(self, planificacion):
        # Evalúa la solución actual contando los días totales que cada actor trabaja.
        días_actor = {actor: 0 for actor in self.df.columns[1:-1]}  # Excluye 'Toma' y 'Total'
        for día in planificacion:
            actores_día = set()
            for toma in día:
                for actor in días_actor:
                    if self.df.iloc[toma-1][actor] > 0:
                        actores_día.add(actor)
            for actor in actores_día:
                días_actor[actor] += 1
        return sum(días_actor.values())

    def generar_vecindad(self, planificacion):
        # Genera vecinos de la solución actual moviendo tomas entre días.
        vecinos = []
        for i in range(len(planificacion)):
            for j in range(i + 1, len(planificacion)):
                for toma in planificacion[i]:
                    if len(planificacion[j]) < self.max_tomas_por_dia:
                        nueva_planificacion = deepcopy(planificacion)
                        nueva_planificacion[i].remove(toma)
                        nueva_planificacion[j].append(toma)
                        vecinos.append(nueva_planificacion)
        return vecinos

    def actualizar_tabu(self, movimiento):
        # Actualiza la lista tabú para incluir movimientos recientes.
        self.tabu_list.append(movimiento)
        if len(self.tabu_list) > self.tamaño_tabu:
            self.tabu_list.pop(0)

    def buscar(self):
        # Realiza la búsqueda tabú para encontrar la mejor solución.
        solución_actual = self.enfoque_voraz()
        mejor_valor = self.función_evaluación(solución_actual)
        self.mejor_solucion = solución_actual

        for _ in range(self.max_iteraciones):
            vecindad = self.generar_vecindad(solución_actual)
            mejor_movimiento = None
            mejor_valor_vecino = float('inf')

            for movimiento in vecindad:
                if movimiento not in self.tabu_list:
                    valor_movimiento = self.función_evaluación(movimiento)
                    if valor_movimiento < mejor_valor_vecino:
                        mejor_movimiento = movimiento
                        mejor_valor_vecino = valor_movimiento

            if mejor_movimiento and mejor_valor_vecino < mejor_valor:
                solución_actual = mejor_movimiento
                mejor_valor = mejor_valor_vecino
                self.actualizar_tabu(mejor_movimiento)
                self.mejor_solucion = solución_actual

        return self.mejor_solucion

# Uso:
planificador = PlanificacionTomas(df, max_tomas_por_dia=6, max_iteraciones=100, tamaño_tabu=10)
mejor_solucion = planificador.buscar()
formato_salida = planificador.formato_solucion(mejor_solucion)
print(formato_salida)



Número total de días de grabación: 5
Día 1: Tomas 1, 11, 12, 4, 10, 22
Día 2: Tomas 26, 20, 25, 7, 6, 9
Día 3: Tomas 3, 13, 5, 15, 2, 29
Día 4: Tomas 8, 14, 24, 28, 27, 16
Día 5: Tomas 23, 21, 19, 18, 17, 30



Respondiendo a las preguntas planteadas podemos distinguir en el código:
- Espacio de Soluciones:

El espacio de soluciones está implícitamente representado por todas las posibles maneras de asignar las tomas a los días dentro de las restricciones dadas. No se representa explícitamente en una estructura de datos, sino que se explora a través de las funciones enfoque_voraz y generar_vecindad.

-Enfoque_voraz: Esta función genera una solución inicial, que es un punto de partida en el espacio de soluciones. La solución es una lista de listas, donde cada sublista representa un día y contiene las tomas asignadas a ese día.

-Generar_vecindad: Esta función explora el espacio de soluciones creando "vecinos" de la solución actual. Estos vecinos son variaciones ligeras de la solución actual, donde una toma se mueve de un día a otro.

- Función objetivo:
La función objetivo en el código es la que se define en el método función_evaluación. Esta función tiene como objetivo minimizar la suma total de los días de trabajo de todos los actores involucrados en las tomas. Se calcula sumando la cantidad de días que cada actor necesita estar presente en el set para las tomas programadas.

- Restricciones:

La restricción de que no se exceda un número máximo de tomas por día está implementada en dos partes del código:

-En enfoque_voraz: Aquí, mientras se distribuyen las tomas entre los días, se verifica si el día actual ya tiene el número máximo de tomas (self.max_tomas_por_dia). Si un día ya tiene el máximo, la toma se asigna al siguiente día. Esto garantiza que no se programen más tomas de las permitidas en un solo día.

-En generar_vecindad: Al crear nuevas soluciones "vecinas", esta restricción también se verifica. Cuando se intenta mover una toma de un día a otro, se asegura de que el día al que se está moviendo la toma no exceda el máximo permitido de tomas.

La búsqueda tabú utiliza una lista tabú para evitar repetir movimientos recientes y así explorar nuevas áreas del espacio de soluciones sin quedar atrapado en ciclos. En actualizar_tabu y en el bucle principal de buscar: Se mantiene una lista tabú (self.tabu_list) que registra los movimientos recientes. Esta lista se utiliza para evitar volver a soluciones recientemente exploradas.

Por último, la reestricción de que todos los actores que participan en la toma deben de estar presentes se lleva a cabo de manera indirecta, cuando se planifican las tomas para cada día, automáticamente se está asegurando que los actores necesarios para esas tomas estén "presentes".

#Análisis
¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

- Orden de Complejidad

-Enfoque Voraz (enfoque_voraz): La complejidad de este método está dominada por la ordenación de las tomas, que es O(nlog⁡n), siendo n el número de tomas.

-Búsqueda Tabú (buscar y generar_vecindad): La generación de vecinos implica crear nuevas configuraciones moviendo tomas entre días. Si asumimos que cada toma puede moverse potencialmente a d−1 días diferentes (en el peor caso), y hay n tomas, la generación de vecinos podría tener una complejidad de hasta O(n×(d−1)) por iteración. Con un número máximo de iteraciones (max_iteracionesmax_iteraciones), la complejidad total sería O(max_iteraciones×n×d).

Sumando ambas partes, la complejidad total sería dominada por la búsqueda tabú, ya que la ordenación en el enfoque voraz es una operación de una sola vez y tiene una complejidad menor en comparación. Por lo tanto, la complejidad computacional total del código se puede aproximar como O(max_iteraciones×n×d).

- Espacio de Soluciones

El espacio de soluciones se refiere a todas las combinaciones posibles de asignar tomas a días. En el peor de los casos, esto es exponencial en el número de tomas, ya que cada toma puede ser asignada a cualquier día. Si tenemos n tomas y potencialmente hasta n días (en el peor caso), el número de maneras de distribuir estas tomas es de orden O(d $^{n}$), siendo d el número de días posibles.

#Diseño
¿Que técnica utilizo? ¿Por qué?

La elección de un algoritmo heurístico, en este caso mediante una técnica  voraz (greedy) para obtener una primera solución y la búsqueda tabú  se basa en varias características clave del problema:

-Complejidad y Tamaño del Problema: El problema tiene un gran espacio de soluciones debido a la cantidad de combinaciones posibles de tomas. Resolver esto de manera exacta por ejemplo, con programación lineal entera es muy costoso en términos de tiempo y recursos computacionales, como aparece en el primer algoritmo que he intentado implementar.

-Eficiencia en Tiempo de Ejecución:La búsqueda Tabú en concreto y los algoritmos heurísticos en general , son conocidos por su eficiencia en tiempo de ejecución. Priorizan la obtención de una solución buena y rápida sobre la búsqueda de la solución óptima, lo que es beneficioso en problemas con muchas variables o cuando se requieren respuestas en tiempos limitados. La búsqueda tabú puede moverse eficientemente a través del espacio de soluciones para encontrar respuestas de alta calidad, evitando quedar atrapada en óptimos locales gracias a su mecanismo de memoria (lista tabú).

-Simplicidad y Facilidad de Implementación: Los enfoques heurísticos suelen ser más sencillos de implementar y entender ya que no requieren de un método matemático en particular.

-Balance entre Intensificación y Diversificación: La búsqueda tabú proporciona un buen equilibrio entre la intensificación (profundizando en áreas del espacio de soluciones donde se han encontrado buenas soluciones, en este caso mediante una técnica voraz) y la diversificación (explorando nuevas áreas del espacio de soluciones para evitar quedar atrapado en soluciones subóptimas).